**Fairness in Hotspot Detection**

This code takes (a) spatial file with sensitive attributes and (b) hotspot file from SaTScan as input

The output is a list of fair hotspots

In [ ]:
#caste heterogenous 

#get_total_caste_distribution

#parsing SaTScan output file

import pandas as pd

def get_clusters(filename):
	f1 = open(filename, "r")
	res_lines = f1.readlines()
	f1.close()
	clusters = []
	
	ids = []
	flag = False
	for l in res_lines:
		if ('Coordinates' in l):
			flag = False
			if (len(ids) != 0):
				clusters.append(ids)
			ids = []
			
		if (flag or 'Location IDs' in l):
			temp = l
			temp = temp[2:]
			temp = temp.replace('Location IDs included.: ', '')
			temp = temp.replace(',\n','').split(', ')

			for i in range(0, len(temp)):
				temp[i] = temp[i].strip()
			ids.extend(temp)

		if ('Location IDs' in l):	
			flag = True
	return clusters

k = 20 #top-k
clusters = get_clusters('result-med-50-new.txt') #SaTScan output file
print(clusters)
print(len(clusters))
clusters = clusters[:k] 

print(len(clusters))
print(clusters)

In [ ]:
#developing caste and religion distribution from dataset
caste_df = pd.read_csv('caste_and_religion_info_income.csv')
print(caste_df)
nf_tot = len(caste_df)
n1 = len(caste_df[caste_df.ID13.astype(int).isin([4,5,6])])
n2 = len(caste_df[caste_df.ID11.astype(int).isin([2,3,4,5,6,7,8,9])])
total = [n1/nf_tot, n2/nf_tot]
print(total)  

In [ ]:
#modules for computing distance between distributions

from scipy.stats import wasserstein_distance

def get_sum(region=''):
  flag = False
  ans = [0]
  if (region == ''):
    return ans
  if (len(region) == 0):
    return ans
  nf_tot = float(len(region))
  if (nf_tot == 0):
    return ans
  n1 = len(caste_df[caste_df.IDPERSON.astype(str).isin(region) & caste_df.ID13.astype(int).isin([4,5,6])])
  n2 = len(caste_df[caste_df.IDPERSON.astype(str).isin(region) & caste_df.ID11.astype(int).isin([2,3,4,5,6,7,8,9])])
  ans = [n1/nf_tot, n2/nf_tot]
  return ans

def div(cluster, total):
	return wasserstein_distance(total, cluster)
 
print(get_sum(clusters[0]))
print(div(get_sum(clusters[0]), total))


In [ ]:
#FiSH Algo

from ast import literal_eval
from random import randrange
from sklearn.preprocessing import StandardScaler
import numpy as np
import copy
import time

#beam
beam_width = 5 #b in the paper
k = 5
l = 0
n = 20 #k in the paper
tau = 5 
d = n * (n+1) / 2

def sort_beam(x):
  new_x = {}
  arr = list(x.values())
  i = 0
  for key, value in x.items():
    flag = False
    for j in range(0, len(arr)):
      if (i == j):
        continue
      if ((arr[i][0] >= arr[j][0]) and (arr[i][1] >= arr[j][1])):
        flag = True
        break
    if (not flag):
      new_x[key] = value
    i += 1
  val_beam = {}
  for key in x.keys():
    val = x[key][0]
    val_beam[key] = val
  val_beam = {k: v for k, v in sorted(val_beam.items(), key=lambda item: item[1])}
  final_beam = {}
  for key in val_beam.keys():
    if (key in new_x):
      final_beam[key] = new_x[key]
  return final_beam




def get_region(areas):
  region = []
  for area in areas:
    region.extend(clusters[area])
  return get_sum(region)
  

beam = {}

start = time.time()
regions = []
begin = div(get_sum(regions), total) 
for i in range(0, len(clusters)):
  region = get_region([i])
  delta = div(region, total)
  vec = [i, delta]
  val = i + l * delta
  a = []
  a.append(i)  
  beam[str(a)] = vec
print(beam)
#sort
beam = sort_beam(beam)
print(beam)


for j in range(1, k):
  print(j)
  new_beam = {}
  for reg in beam.keys():
    area = literal_eval(reg)
    for i in range(0, len(clusters)):	
      if str(i) in reg:
        continue
      new_area = copy.deepcopy(area)
      new_area.append(i)
      list.sort(new_area)
      new_reg = str(new_area)
      reg_consd = get_region(new_area)
      delta = div(reg_consd, total)
      t = sum(new_area)
      vec = [t, delta]
  
      new_beam[new_reg] = vec
      new_beam = sort_beam(new_beam)
  final_beam = {}
  cand = list(new_beam.keys())
  cand_length = len(cand)
  fraction = 1 / (beam_width - 1)
  final_beam[cand[0]] = new_beam[cand[0]]
  for m in range(1, beam_width):
    ind = int(m * fraction * cand_length) - 1
    print(ind)
    final_beam[cand[ind]] = new_beam[cand[ind]]
  final_beam[cand[cand_length-1]] = new_beam[cand[cand_length-1]]
  beam = sort_beam(final_beam)
  print(beam)
        
print(beam)
print(len(beam))
final_beam = {}
cand = list(beam.keys())
cand_length = len(cand)
fraction = 1 / (tau - 1)
final_beam[cand[0]] = new_beam[cand[0]]
for m in range(1, tau):
  ind = int(m * fraction * cand_length) - 1
  #ind = m
  print(ind)
  final_beam[cand[ind]] = new_beam[cand[ind]]
final_beam[cand[cand_length-1]] = new_beam[cand[cand_length-1]]
beam = sort_beam(final_beam) #fair hotspots
print(beam)
print(len(beam))
print(time.time() - start)

In [ ]:
#computing skyline for the purpose of evaluation

skyline = {}
all = []

import time

start = time.time()
m = len(clusters)
for a in range(0, m):
  for b in range(a+1, m):
    for c in range(b+1, m):
      for d in range (c+1, m):
        for e in range(d+1, m):
          ind = [a, b, c, d, e]
          region = []
          for i in range(0, 5):
            region.extend(clusters[ind[i]])
          delta = div(get_sum(region), total)
          t = a+b+c+d+e
          all.append([t, delta])
          if (len(skyline) == 0):
            skyline[str(ind)]=[t, delta]
          else:
            flag = False
            replace = []
            count = 0
            for key, item in skyline.items():
              if (item[0] >= t and item[1] >= delta):
                flag = True
                replace.append(key)
              if ((t < item[0] and delta >= item[1]) or (t >= item[0] and delta < item[1])):
                count += 1
            if (count == len(skyline)):
              flag = True
            if (flag):
              skyline [str(ind)] = [t, delta]
              if (len(replace) != 0):
                for r in replace:
                  del(skyline[r])
           
print(skyline)
print(len(skyline))
print(all)
print(time.time()-start)

In [ ]:
#scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
all_data = scaler.fit_transform(all_clusters)


In [ ]:
#plotting results
import matplotlib.pyplot as plt

#all points
all_clusters = np.asarray(all_clusters)
plt.plot(all_clusters[:,0], all_clusters[:,1],'ro')
plt.show()

In [ ]:
all_data = np.asarray(all_data)
plt.plot(all_data[:,1], all_data[:,0],'ro')
plt.show()

In [ ]:
#skyline
xa = []
ya = []
arr_sky = []
for val in skyline.values():
  xa.append(val[0])
  ya.append(val[1])
  arr_sky.append([val[0], val[1]])

arr_sky = np.asarray(arr_sky)
arr_sky_norm = scaler.transform(arr_sky)

plt.plot(arr_sky_norm[:,1], arr_sky_norm[:,0],'bo')
plt.show()


In [ ]:
#print beam points
import matplotlib.pyplot as plt

x2 = []
y2 = []
arr_beam = []

for item in beam.keys():
  area = literal_eval(item)
  x2.append(div(get_region(area), total))
  y2.append(sum(area))
  arr_beam.append([sum(area), div(get_region(area), total)])

arr_beam = np.asarray(arr_beam)
arr_beam_norm = scaler.transform(arr_beam)

plt.plot(arr_beam_norm[:,1], arr_beam_norm[:,0],'go')
plt.show()


In [ ]:
plt.plot(all_data[:,1], all_data[:,0],'ro')
jittered_y = arr_sky_norm[:,0] + 0.015 * np.random.rand(len(arr_sky_norm[:,0]))
plt.plot(arr_sky_norm[:,1], jittered_y,'bo', alpha=0.7)
plt.plot(arr_beam_norm[:,1], arr_beam_norm[:,0],'go')
plt.show()


In [ ]:
#gold standard
beam_width = len(beam)
fraction = 1 / (beam_width - 1)
gold_standard = [arr_sky_norm[0]]
for m in range(1, beam_width):
  ind = int(m * fraction * len(arr_sky_norm)) - 1
  #ind = m
  gold_standard.append(arr_sky_norm[ind])

print(len(arr_sky_norm))
print(gold_standard)
print(arr_beam_norm)
gold_standard = np.asarray(gold_standard)
print(gold_standard)

#plt.plot(arr_sky_norm[:,0], arr_sky_norm[:,1],'bo')
plt.plot(gold_standard[:,0], gold_standard[:,1],'yo')
plt.show()
plt.plot(arr_beam_norm[:,0], arr_beam_norm[:,1],'go')
plt.show()

In [ ]:
jittered_y = arr_beam_norm[:,0] + 0.018 * np.random.rand(len(arr_beam_norm[:,0]))
jittered_y_gold = gold_standard[:,0] - 0.02 * np.random.rand(len(gold_standard[:,0]))
plt.plot(all_data[:,1], all_data[:,0],'ro',alpha=0.8, markersize=3)
plt.plot(gold_standard[:,1], jittered_y_gold,'yo', markersize=9, mec='black')
plt.plot(arr_beam_norm[:,1], jittered_y,'go', alpha=0.8, markersize=9, mec='black')
plt.xlim(-0.005,0.125)
plt.ylim(-0.05, 0.8)
plt.xlabel('Deviation')
plt.ylabel('Sum of Ranks')
plt.tick_params(
    axis='both',         
    which='both',          
    bottom=False,   
    top=False,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False) 
plt.show()

In [ ]:
#direct comparison
import math

dc = 0
for i in range(0, len(beam)):
  cur_dist = math.sqrt((arr_beam_norm[i,0]-gold_standard[i,0])**2 
                       + (arr_beam_norm[i,1]-gold_standard[i,1])**2)
  dc += cur_dist
print(dc)

In [ ]:
#coverage

n = len(all_data)
print(n)

#coverage-beam
def coverage(candidate, corpus):
  n = len(corpus)
  count = 0.0
  for i in range(0, n):
    for j in range(0, len(candidate)):
      x = corpus[i]
      y = candidate[j]
      if (y[0] < x[0] and y[1] < x[1]):
        count += 1
        break
  return count / n

print(coverage(arr_beam_norm, all_data))
    

In [ ]:
#diversity

from sklearn.metrics.pairwise import euclidean_distances
#dist1 = euclidean_distances(gold_standard, gold_standard)
dist2 = euclidean_distances(arr_beam_norm, arr_beam_norm)
#print(dist1)
print(dist2)

#average1 = dist1[np.nonzero(dist1)].min()
average2 = dist2[np.nonzero(dist2)].min()
print(average2)
#print(average1)